In [7]:
from EAC import site
import os 
import pandas as pd
import sqlite3

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
training = site.training()

conn = sqlite3.connect('training.db')

In [12]:
pd.read_sql('''
select * from movements
''',conn)        
            

,movement,split,acutalMax,repMaxCalc,veloEquation,kilos
0,bench,upper/chest/push,157.5,None,Missing,1
1,squat,lower/legs,262.5,None,Missing,1
2,convo,back/pull,280.0,None,Missing,1
3,sumo,back/pull,285.0,None,Missing,1
4,trap,back/pull,308.0,None,Missing,1
...,...,...,...,...,...,...
69,incline db curl,upper/bicep/pull,0.0,None,Missing,0
70,incline concentration curl db,upper/bicep/pull,0.0,None,Missing,0
71,incline concentration curl kab,upper/bicep/pull,0.0,None,Missing,0
72,incline concentration curl ez,upper/bicep/pull,0.0,None,Missing,0


In [13]:
pd.read_sql('''
select * from upcomingTraining
''',conn)   

,week,session,movement,method,sets,reps,weight,progNotes,targetReps


In [20]:
pd.read_csv(dataPath+trainingFile)

,endWeek,session,ordering,exercise,modality,SetAdjust
0,9,Push1,1,bench,wendDouble,NaN
1,9,Push1,2,board bench,9str,NaN
2,9,Push1,3,BandBell Incline,Grow,NaN
3,9,Legs1,1,Squat,wendDouble,NaN
4,9,Legs1,2,Tsunami Back,Build,NaN
5,9,Legs1,3,Tsunami Front,Build,NaN
6,9,Pull1,1,Deadlift,wendDouble,NaN
7,9,Pull1,2,Trap,3Back,NaN
8,9,Pull1,3,Meadows,9hyp,1.0
9,9,Push2,2,Bandbell press,Build,1.0


In [14]:
pd.read_sql('''
select * from programBuilder
''',conn)   

,methodID,method,week,sets,reps,weight,notes,targetReps
0,5x531_1,5x531,1,5,5,76.5,,9
1,5x531_2,5x531,2,5,3,81.0,,7
2,5x531_3,5x531,3,5,1,85.5,,5
3,wendDouble_1,wendDouble,1,2,1,82.5,cluster doubles/required work/drop 10% rep,
4,wendDouble_2,wendDouble,2,2,1,85.0,go up five %/required/hard triple/clusters,
...,...,...,...,...,...,...,...,...
129,contrast_1,contrast,1,6,1 6 1 6 1 6+,90 75 90 77.5 90 80,,
130,highWave_1,highWave,1,6,7 5 3 7 5 3,75 80 82.5 +10 +10 +10,,
131,50s_1,50s,1,x,50,70.0,as many sets to get to 50 time,
132,back-offs_1,back-offs,1,4,6-15,,work up to rep max and then take 90% of that a...,


In [ ]:
create_meso_q = '''
CREATE TABLE upcomingTraining (
week int,
session char(20),
movement char(20),
method char(20),
sets int,
reps int,
weight float(4),
progNotes char(140),
targetReps int
)
'''

In [ ]:
#each movement/lift is the pk
create_movement_table_q = '''
CREATE TABLE movements (
movement CHAR(40) PRIMARY KEY not null,
split CHAR(10),
acutalMax int,
repMaxCalc float(4),
veloEquation CHAR(30),
kilos binary
);

'''

In [ ]:
#methodID is a amalgamation of method and week has 8 columns
create_program_builder_q = '''
CREATE TABLE programBuilder (
methodID char(30) PRIMARY KEY not null,
method char(20),
week int,
sets int,
reps int,
weight float(4),
notes char(140),
targetReps int
);
'''

In [ ]:
#sessionID  = date_exercise
create_history_q = '''
CREATE TABLE history(
sessionID PRIMARY KEY not null,
sessionDate date,
session char(30),
movement CHAR(40),
method CHAR(20),
setsCompleted int,
repsCompleted int,
weightUsed int,
amrapTotal int,
newMax binary,
notes CHAR(200),
RepCalcM int);
'''

In [ ]:
#veloID = date_exercise_set_rep
create_velo_q='''
CREATE TABLE velo (
movement char(40),
sessionDate date,
weight_kg float(4),
weight_lb float(4),
set_ct int,
rep int,
ecc_con char(4),
MPV_ms float(4),
peak_velo float(4),
rom_cm float(2),
rom_in float(2),
watts int,
est_max_kg float(4),
est_max_lb float(4)


)
'''

In [ ]:
create_rpe_q = '''
CREATE TABLE rpe (
RPE float(1) PRIMARY KEY not null,
rep1 float(4),
rep2 float(4),
rep3 float(4),
rep4 float(4),
rep5 float(4),
rep6 float(4),
rep7 float(4),
rep8 float(4),
rep9 float(4),
rep10 float(4),
rep11 float(4),
rep12 float(4),
rep13 float(4),
rep14 float(4),
rep15 float(4),
rep16 float(4)
);'''

In [ ]:

conn.execute(create_program_builder_q)
# conn.execute(create_meso_q)
# conn.execute(create_rpe_q)
# conn.execute(create_history_q)
# conn.execute(create_movement_table_q)
# conn.execute(create_velo_q)
conn.commit()


### Data Loading

In [ ]:
mesoFile = [f for f in files if f.find(' Meso')!=-1][0]
startDate = mesoFile.split(' - ')[0][-8:]
weeks = mesoFile.split(' - ')[1][9:11]
setup = mesoFile.split(' - ')[1][12:-4]


In [46]:
def create_meso_cycle(path,connection):
    df = pd.DataFrame(columns = ['movement','method','week','sets','reps','weight','notes','targetReps'])
    prgbld = pd.read_sql('''
    select * 
    from programBuilder
    ''',conn)
    mesoDF = pd.read_csv(path).fillna(0)
    v = df.columns[-5:].values
    r=0
    for sesh in mesoDF.session.unique():
        d = mesoDF[mesoDF.session == sesh]
        for movement in d.exercise.unique():
            endweek = d[d.exercise==movement].endWeek.values[0]
            mod = d[d.exercise==movement].modality.values[0]
            cyclelength = prgbld[prgbld.method==mod].week.max()
            setAdjust = d[d.exercise==movement].SetAdjust.values[0]
            for i in range(endweek):
                lkup = i+1 if cyclelength==endweek else (i+1) % cyclelength
                lkup = cyclelength if lkup == 0 else lkup
                l = [movement,mod,i+1]+prgbld[(prgbld.method==mod) & (prgbld.week==lkup)].filter(v).values.tolist()[0]
                df.loc[len(df)] = l
                df.at[r,'sets'] = df.at[r,'sets'] + setAdjust
                r+=1
    path = os.getcwd()+'/initialData/meso_file.csv'
    df.to_csv(path)
    return df,path

In [50]:
def add_data(path,table,connection):
    with open(path,'r') as d:
        for i,row in enumerate(d):
            if i==0:
                continue
            else:
                row = row.replace('\n','')
                qs = (len(row.split(','))-1)*'?,'+'?'
                connection.execute('''INSERT into {t} values ({row})'''.format(t=table,row=qs),row.split(','))
        connection.commit()
    return '{} has been updated with {} rows of data'.format(table,i)


In [15]:
import os
dataPath = os.getcwd()+'/initialData/'

In [16]:
pgFile = 'MesoLoad_06022023 - ProgramBuilder.csv'
moveFile = 'MesoLoad_06022023 - lifts.csv'
rpeFile='MesoLoad_06022023 - RPE.csv'
trainingFile = 'MesoLoad_10132023 - MesoLoad_18_PPL.csv'

In [ ]:
conn.execute('DROP TABLE programBuilder')
conn.execute(create_program_builder_q)
conn.commit()

In [49]:
meso,mpath = create_meso_cycle(dataPath+trainingFile,conn)
add_data(mpath,'upcomingTraining',conn)

0,bench,wendDouble,1,2,1,82.5,cluster doubles/required work/drop 10% rep,

1,bench,wendDouble,2,2,1,85.0,go up five %/required/hard triple/clusters,

2,bench,wendDouble,3,2,1,87.5,go up 5%/clusters,

3,bench,wendDouble,4,2,1,82.5,cluster doubles/required work/drop 10% rep,

4,bench,wendDouble,5,2,1,85.0,go up five %/required/hard triple/clusters,

5,bench,wendDouble,6,2,1,87.5,go up 5%/clusters,

6,bench,wendDouble,7,2,1,82.5,cluster doubles/required work/drop 10% rep,

7,bench,wendDouble,8,2,1,85.0,go up five %/required/hard triple/clusters,

8,bench,wendDouble,9,2,1,87.5,go up 5%/clusters,

9,board bench,9str,1,5,5,72.5,,9

10,board bench,9str,2,5,4,77.5,,7

11,board bench,9str,3,5,3,82.5,,5

12,board bench,9str,4,5,4,77.5,,7

13,board bench,9str,5,5,3,82.5,,5

14,board bench,9str,6,5,2,87.5,,3

15,board bench,9str,7,5,3,82.5,,5

16,board bench,9str,8,5,2,87.5,,3

17,board bench,9str,9,5,1,92.5,,2

18,BandBell Incline,Grow,1,4,15,60.0,,17

19,BandBell Incline,Grow,2,4,12,65.0,,14

20

'upcomingTraining has been updated with 135 rows of data'

In [18]:
from EAC import database

In [19]:
db = database()

In [32]:
meso

,movement,method,week,sets,reps,weight,notes,targetReps
0,bench,wendDouble,1,2,1,82.5,cluster doubles/required work/drop 10% rep,
1,bench,wendDouble,2,2,1,85.0,go up five %/required/hard triple/clusters,
2,bench,wendDouble,3,2,1,87.5,go up 5%/clusters,
3,bench,wendDouble,4,2,1,82.5,cluster doubles/required work/drop 10% rep,
4,bench,wendDouble,5,2,1,85.0,go up five %/required/hard triple/clusters,
...,...,...,...,...,...,...,...,...
130,Seal,Grow,5,5,11,67.5,,13
131,Seal,Grow,6,5,8,72.5,,11
132,Seal,Grow,7,5,13,65.0,,14
133,Seal,Grow,8,5,10,70.0,,12


In [ ]:
db.create_meso_cycle(dataPath+trainingFile)

In [31]:
df

NameError: name 'df' is not defined